In [ ]:
#Import Packages needed
from os import path
import pandas as pd
from simple_salesforce import Salesforce
import numpy as np
import os
import datetime as datetime
import pyarrow as pa
from datetime import date, datetime, timedelta
from functools import reduce

user = "corey@corey-rastello.com"
pw = "your_password"
token = "your_salesforce_security_token"

#connect to sf
sf = Salesforce(username=user, password=pw, security_token=token)

def sf_api_query(data):
 df = pd.DataFrame(data['records']).drop('attributes', axis=1)
 listColumns = list(df.columns)
 for col in listColumns:
     if any (isinstance (df[col].values[i], dict) for i in range(0, len(df[col].values))):
         df = pd.concat([df.drop(columns=[col]),df[col].apply(pd.Series).drop('attributes',axis=1).add_prefix(col+'.')],axis=1)
         new_columns = np.setdiff1d(df.columns, listColumns)
         for i in new_columns:
             listColumns.append(i)
 return df

In [ ]:
stagelookup = pd.read_csv('StageLookups.csv')
lookup_dict = dict(zip(stagelookup['StageName'], stagelookup['Lookup']))

In [3]:
soql = """
        SELECT
        FIELDS(ALL)
        FROM OpportunityStage
        LIMIT 200
"""

# We can Execute the query via python
# or we can figure a way to update the Azure SQL DB soql column
stages = sf_api_query(sf.query_all(soql))
stages = stages[['MasterLabel','IsWon','IsClosed']]
stages = stages.rename(columns={'MasterLabel':'StageName'})
stages.tail()

In [ ]:
# We can then create a SOQL String
# SOQL Query for Accounts
soql = """
        SELECT
        Id,
        NewValue, 
        OldValue, 
        Field, 
        CreatedDate, 
        OpportunityId

        FROM OpportunityFieldHistory

        WHERE Field IN ('ARR_Amount__c','StageName','CloseDate','created','NextStep')
        AND CreatedDate >= LAST_N_FISCAL_YEARS:3
"""

# We can Execute the query via python
fieldHistory = sf_api_query(sf.query_all(soql))

fieldHistory= fieldHistory.sort_values(by=['Id'],ascending=True)

fieldHistory.drop_duplicates(
    subset=["OpportunityId", "Field", "CreatedDate"], keep="last", inplace=True
)

fieldHistory["CreatedDate"] = (
    pd.to_datetime(fieldHistory["CreatedDate"])
    .dt.tz_convert("America/Chicago")
    .dt.tz_localize(None)
)


In [ ]:
# SOQL Query for Accounts
soql = """
        SELECT
        Id,
        ARR_Amount__c,
        StageName,
        CloseDate,
        CreatedDate,
        CurrencyIsoCode,
        NextStep

        FROM Opportunity

"""

# We can Execute the query via python
# or we can figure a way to update the Azure SQL DB soql column
opps = sf_api_query(sf.query_all(soql))
opps.rename(columns={"CreatedDate": "created"}, inplace=True)

currency = pd.read_csv("./currency.csv")

oppcurrency = opps[["Id", "CurrencyIsoCode"]]
oppcurrency.columns = ["OpportunityId", "CurrencyIsoCode"]

created = opps[["Id", "created"]]
created.columns = ["Id", "CreatedDate"]

# melt/transpose the opps

opps = pd.melt(
    opps,
    id_vars=["Id"],
    value_vars=[
        "ARR_Amount__c",
        "StageName",
        "CloseDate",
        "created",
        "NextStep"
    ],
    var_name="Field",
    value_name="NewValue",
)

opps = pd.merge(opps, created, how="left", on="Id")

opps["CreatedDate"] = (
    pd.to_datetime(opps["CreatedDate"])
    .dt.tz_convert("America/Chicago")
    .dt.tz_localize(None)
)

In [7]:
fieldVals = pd.DataFrame(
    fieldHistory.groupby(["OpportunityId", "Field"])["CreatedDate"].nunique()
).reset_index()
fieldVals = fieldVals[["OpportunityId", "Field"]]
fieldVals.columns = ["Id", "Field"]

In [8]:
oppValsToAdd = pd.merge(opps, fieldVals, how="outer", indicator=True)
oppValsToAdd = oppValsToAdd[oppValsToAdd["_merge"] == "left_only"]
oppValsToAdd = oppValsToAdd.drop(columns="_merge")
oppValsToAdd.rename(columns={"Id": "OpportunityId"}, inplace=True)
oppValsToAdd

In [9]:
OppFieldHistory = pd.concat([oppValsToAdd, fieldHistory])
OppFieldHistory = OppFieldHistory.fillna("None")
OppFieldHistory

In [10]:
FirstOldValue = (
    OppFieldHistory.sort_values("CreatedDate")
    .groupby(["OpportunityId", "Field"])
    .first()
    .reset_index()
)
createdate = FirstOldValue.groupby(["OpportunityId"])["CreatedDate"].min()
FirstOldValue["CreatedDate"] = FirstOldValue["OpportunityId"].apply(
    lambda x: createdate[x]
)
FirstOldValue = FirstOldValue[
    (((FirstOldValue["Field"] == "StageName") & (FirstOldValue["OldValue"] == "0")) | (FirstOldValue["OldValue"] != "0")) & (FirstOldValue["OldValue"] != "None")
]
FirstOldValue["NewValue"] = FirstOldValue["OldValue"]
FirstOldValue["OldValue"] = "None"
FirstOldValue

In [11]:
OppFieldHistory = pd.concat([OppFieldHistory, FirstOldValue])
OppFieldHistory.drop_duplicates(
    subset=["OpportunityId", "Field", "CreatedDate"], keep="first", inplace=True
)
OppFieldHistory.sort_values(by='CreatedDate',inplace=True)
OppFieldHistory.head()

In [12]:
OppFieldHistory = OppFieldHistory.pivot_table(
    index=["OpportunityId", "CreatedDate"],
    values=["NewValue"],
    columns="Field",
    aggfunc="first",
)
OppFieldHistory = OppFieldHistory.droplevel(0, axis=1).reset_index()
OppFieldHistory.head()

In [13]:
for col in OppFieldHistory.columns:

    OppFieldHistory = OppFieldHistory.sort_values(
        by=["OpportunityId", "CreatedDate"], ascending=[True, True]
    )

    OppFieldHistory[col] = (
        OppFieldHistory.groupby(OppFieldHistory["OpportunityId"])[col]
        .ffill()
        .infer_objects(copy=False)
    )


OppFieldHistory = pd.merge(OppFieldHistory, oppcurrency, how="left", on="OpportunityId")


OppFieldHistory = pd.merge(OppFieldHistory, currency, how="left", on="CurrencyIsoCode")


OppFieldHistory.head(10)

In [ ]:
valFields = ["ARR_Amount__c"]

for col in valFields:
    OppFieldHistory[col] = OppFieldHistory[col].replace("None", 0)
    OppFieldHistory[col] = OppFieldHistory[col].fillna(0)
    OppFieldHistory[col] = OppFieldHistory[col].astype("Float32")
    OppFieldHistory[col] = OppFieldHistory[col] / OppFieldHistory["Rate"]


OppFieldHistory.head()

In [ ]:
OppFieldHistory = OppFieldHistory[
    [
        "OpportunityId",
        "CreatedDate",
        "ARR_Amount__c",
        "CloseDate",
        "StageName",
        "NextStep"
    ]
]

OppFieldHistory['StageName'] = OppFieldHistory['StageName'].replace(lookup_dict)

OppFieldHistory.head()

In [ ]:
soql = """
        SELECT
        Id,
        ACV_USD_Fixed_Rate_New__c,
        Name,
        ForecastCategory,
        StageName,
        CloseDate,
        CreatedDate,
        CurrencyIsoCode,
        Exclude_from_reporting__c,
        IsClosed,
        IsWon,
        Test_Account__c,
        Account.Test_Account__c



        FROM Opportunity

"""

# We can Execute the query via python
opps = sf_api_query(sf.query_all(soql))

opps["CreatedDate"] = (
    pd.to_datetime(opps["CreatedDate"])
    .dt.tz_convert("America/Chicago")
    .dt.tz_convert("America/Chicago")
    .dt.tz_localize(None)
)

opps.to_parquet('OpportunityFieldHistoryOpps.parquet',index=False)

In [ ]:
OppFieldHistory['Opportunity History Sort Key'] = OppFieldHistory['CreatedDate'].dt.strftime('%Y%m%d%H%M%S') + OppFieldHistory['OpportunityId'].astype(str)

# Optimize the calculation of ValidToDate by pre-sorting the DataFrame
OppFieldHistory = OppFieldHistory.sort_values(by=['OpportunityId', 'Opportunity History Sort Key'])

# Calculate ValidToDate by shifting the CreatedDate within each OpportunityId group
OppFieldHistory['ValidToDate'] = OppFieldHistory.groupby('OpportunityId')['CreatedDate'].shift(-1)

# Calculate Is Last Update
OppFieldHistory['Is Last Update'] = OppFieldHistory['ValidToDate'].isna().astype(int)

print(len(OppFieldHistory))
OppFieldHistory = pd.merge(OppFieldHistory,stages, how='left',on='StageName')
print(len(OppFieldHistory))

print(len(OppFieldHistory))
OppFieldHistory = pd.merge(OppFieldHistory,opps[['Id','CreatedDate']].rename(columns={'Id':'OpportunityId','CreatedDate':'Opportunity Created Date'}), how='left',on='OpportunityId')
print(len(OppFieldHistory))

OppFieldHistory['Stage IsLost'] = OppFieldHistory['StageName'] == "Closed Lost"
OppFieldHistory['Stage IsDQ'] = OppFieldHistory['StageName'] == "Disqualified"
OppFieldHistory['Amount USD'] = OppFieldHistory['ARR_Amount__c']

OppFieldHistory = OppFieldHistory.rename(columns={'ARR_Amount__c':'AmountUSD'})

OppFieldHistory.head()

In [ ]:
OppFieldHistory = OppFieldHistory.groupby('OpportunityId').filter(lambda x: not all(x['AmountUSD'] == 0))

In [ ]:
OppFieldHistory.to_parquet('OpportunityFieldHistory.parquet',index=False)